In [4]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
tfk = tf.keras
tfkl = tfk.layers
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.datasets import cifar10
import tensorflow_probability as tfp#tensorflow_probabilityは長いのでtfpとして簡略化してimport
#分布の省略も準備する
tfd = tfp.distributions
tfpl = tfp.layers

np.random.seed(0)
tf.random.set_seed(0)
#データセットの準備
import tensorflow_datasets as tfds
datasets, datasets_info = tfds.load(name='cifar10', with_info=True,
                                    as_supervised=False)

def _preprocess(sample):
  image = tf.cast(sample['image'], tf.float32) / 255.  # Scale to unit interval.
  image = image < tf.random.uniform(tf.shape(image))   # Randomly binarize.
  return image, image

train_dataset = (datasets['train']
                 .map(_preprocess)
                 .batch(256)
                 .prefetch(tf.data.experimental.AUTOTUNE)
                 .shuffle(int(10e3)))
eval_dataset = (datasets['test']
                .map(_preprocess)
                .batch(256)
                .prefetch(tf.data.experimental.AUTOTUNE))

encoded_size = 16 #圧縮時のサイズを設定
base_depth = 32 #フィルター数
#学習パラメータのない独立した平均0,分散1の正規分布乱数をencoded_size個作成
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

encoder = models.Sequential([
    tfkl.InputLayer(input_shape= (32, 32, 3)),
    #tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    #今回は画像の畳み込みを使用するに当たって
    #一般に画像はRGB画像のように、RGBの3つのレイヤーを持つので、(縦, 横, レイヤー)の
    #入力に変換する必要がある。
    #今回のグレースケール画像なら、余分にレイヤーを加え、(28,28,1)の形式に変換する
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    #【layers.Conv2Dの説明】
    #base_depthはフィルターで生成される画像の数
    #5の引数は畳み込みをするフィルターのサイズ
    #stride はピクセル間の隙間のピクセル数を制御
    #conv1では（28,28,2）が出力になる。
    #これは1枚の入力画像に対して、28×28ピクセルの画像が2つ出力されることを意味する
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),#多変量正規分布の混合
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
])
#エンコーダと事前確率との間の KL ダイバージェンスを正則化項として追加する
#weightを 1 以外のものに変えるだけで、この VAE を β-VAE に変更できる
encoder.summary()

decoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                         padding='valid', activation=tf.nn.leaky_relu),
                          # (1,1,16)->(7,7,64)に膨らむ
                          # base_depth = 32 はフィルター数
                          # 出力画像サイズは7
                          # padding がvalid ならパディングをしない
                          # output_shape = (input_shape-1)×strides+filter_shape=(1-1)×1+7
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
                          # padding が same でゼロパディング
                          # output_shape = input_shape×strides=7×1
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                padding='same', activation=None),
    tfkl.Flatten(),
    tfpl.IndependentBernoulli((32, 32, 3), tfd.Bernoulli.logits),
])
#ピクセル間で独立したベルヌーイ分布にする
decoder.summary()

vae = tfk.Model(inputs=encoder.inputs,
                outputs=decoder(encoder.outputs[0]))
vae.summary()

negloglik = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negloglik)

history = vae.fit(train_dataset, epochs=10, validation_data=eval_dataset)

def display_imgs(x, title=None, fignum=None):
  import matplotlib.pyplot as plt  # pylint: disable=import-outside-toplevel,g-import-not-at-top
  if not tf.executing_eagerly():
    raise NotImplementedError('`display_imgs` can only be executed eagerly.')
  def _preprocess(z):
    return np.array(getattr(z, 'numpy', lambda: z)())
  x = _preprocess(x)
  if title is not None:
    title = _preprocess(title)
  x = np.reshape(x, (-1,) + x.shape[-4:])
  nrows, ncols, h, w, c = x.shape
  x = np.reshape(np.transpose(x, [0, 2, 1, 3, 4]), [nrows * h, ncols * w, c])
  plt.ioff()
  subplots_kwargs = dict(
      nrows=1,
      ncols=1,
      figsize=(ncols, max(2, nrows)),
      num=fignum,
      clear=True)
  try:
    fig, axs = plt.subplots(**subplots_kwargs)
  except TypeError:
    subplots_kwargs.pop('clear')
    fig, axs = plt.subplots(**subplots_kwargs)
  axs.imshow(x.squeeze(), interpolation='none')
  axs.axis('off')
  if title is not None:
    axs.set_title(str(title))
  fig.tight_layout()
  plt.show()
  plt.ion()
  return fig, axs

  x = next(iter(eval_dataset))[0][:10]
xhat = vae(x)
assert isinstance(xhat, tfd.Distribution)
print('元の画像:')
display_imgs(x)

print('デコーダの乱数:')
display_imgs(xhat.sample())

print('デコーダ最頻値:')
display_imgs(xhat.mode())

print('デコーダ平均値:')
display_imgs(xhat.mean())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 32, 32, 32)        2432      
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 16, 32)        25632     
                                                                 
 conv2d_19 (Conv2D)          (None, 16, 16, 64)        51264     
                                                                 
 conv2d_20 (Conv2D)          (None, 8, 8, 64)          102464    
                                                                 
 conv2d_21 (Conv2D)          (None, 2, 2, 64)          200768    
                                                                 
 flatten_5 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 152)              

InvalidArgumentError: ignored